In [1]:
import sys

sys.path.insert(0, '/home/jianx/search-exposure/')
import torch
from annoy import AnnoyIndex
import forward_ranker.load_data as load_data
import forward_ranker.train as train
from forward_ranker.utils import print_message
from forward_ranker.test import get_ndcg_precision_rr
obj_reader = load_data.obj_reader
obj_writer = load_data.obj_writer
import pickle
import numpy as np
import pandas as pd
import random
import math
import  matplotlib.pyplot as plt

In [2]:
vocabs = {}
with open("/datadrive/jianx/data/queries.train.tsv","r") as f:
    for line in f:
        qid = line.split("\t")[0]
        query = line.split("\t")[1]
        for word in query.split(" "):
            if word not in vocabs:
                vocabs[word] = 0
            vocabs[word] += 1

In [18]:
query_length_dict = {}
with open("/datadrive/jianx/data/queries.train.tsv","r") as f:
    for line in f:
        qid = int(line.split("\t")[0])
        query = line.split("\t")[1]
        query_length_dict[qid] = len(query.split(" "))

In [12]:
query_term_freq_dict = {}
with open("/datadrive/jianx/data/queries.train.tsv","r") as f:
    for line in f:
        qid = int(line.split("\t")[0])
        query = line.split("\t")[1]
        freq = 0
        for word in query.split(" "):
            freq += vocabs[word]
        query_term_freq_dict[qid] = freq

In [20]:
baseline_full = obj_reader("/datadrive/ruohan/reverse_ranker/residual/loss15_layer3/forward_baseline_rank_test.pickle")
append_full = obj_reader("/datadrive/ruohan/reverse_ranker/append/layer3_no_split/pred_rank_test.pickle")
residual_full = obj_reader("/datadrive/ruohan/reverse_ranker/residual/loss15_layer3/pred_rank_test.pickle")

In [8]:
all_results = {}
with open("/datadrive/jianx/data/results/all_search_rankings_100_100_flat.csv", "r") as f:
    for line in f:
        qid = int(line.split(",")[0])
        pid = int(line.split(",")[1])
        rank = int(line.split(",")[2])
        if pid not in all_results.keys():
            all_results[pid] = {}
        all_results[pid][qid] = rank

In [15]:
def get_match_mis_length_freq(true_dict, test_dict):
    pids = list(test_dict.keys())
    matched_freq = 0
    matched_len = 0
    num_matches = 0
    missed_freq = 0
    missed_len = 0
    num_misses = 0
    for pid in pids:
        if pid in true_dict:
            for qid in test_dict[pid]:
                if qid in true_dict[pid]:
                    matched_freq += query_term_freq_dict[qid]/query_length_dict[qid]
                    matched_len  += query_length_dict[qid]
                    num_matches += 1
            for qid in true_dict[pid]:
                if qid not in test_dict[pid]:
                    missed_freq += query_term_freq_dict[qid]/query_length_dict[qid]
                    missed_len  += query_length_dict[qid]
                    num_misses += 1
    print("Matched avg frequency:{} Missed avg frequency:{}".format(matched_freq/num_matches, missed_freq/num_misses))
    print("Matched avg length:{} Missed avg length:{}".format(matched_len/num_matches, missed_len/num_misses))

In [21]:
print("Baseline")
get_match_mis_length_freq(all_results, baseline_full)
print("Append")
get_match_mis_length_freq(all_results, append_full)
print("Residual")
get_match_mis_length_freq(all_results, residual_full)

Baseline
Matched avg frequency:61878.203359315325 Missed avg frequency:60585.50024997981
Matched avg length:6.093338178553488 Missed avg length:5.45666029318037
Append
Matched avg frequency:60952.804382117785 Missed avg frequency:76191.42379751215
Matched avg length:6.029226505416982 Missed avg length:5.114575363427697
Residual
Matched avg frequency:60874.24922742646 Missed avg frequency:78419.13318750002
Matched avg length:6.031645038985719 Missed avg length:5.029738302934179
